#  Historical Timeline Preprocessing 

---

<a name="1"></a>
# *Loading Libraries* 
---

In [1]:
import re
import json

import pandas as pd
import requests
from bs4 import BeautifulSoup
from deep_translator import GoogleTranslator

---

<a name="1"></a>
# *Extract Data from [wikipedia page](https://fr.wikipedia.org/wiki/Jeux_olympiques)* 
---

In [118]:
# URL of the Wikipedia page
url = "https://fr.wikipedia.org/wiki/Jeux_olympiques"

# Sending an HTTP request to retrieve the page content
response = requests.get(url)
response.raise_for_status()  

# Parsing the HTML content with BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Finding all tables with class "wikitable"
tables = soup.find_all('table', {"class": "wikitable"})

def extract_table(table):
    # Extracting table headers
    headers = [header.text.strip() for header in table.find_all('th')]
    # Extracting table rows
    rows = table.find_all('tr')[1:]
     # Extracting data from table cells
    data = [[cell.text.strip() for cell in row.find_all(['td', 'th'])] for row in rows]
    return pd.DataFrame(data, columns=headers)

# Extracting data from the first two tables
tables_data = []
for i, table in enumerate(tables):
    try:
        df = extract_table(table)
        tables_data.append(df)
    except Exception as e:
        print(f"Erreur lors de l'extraction du tableau {i}: {e}")
    if i == 1:
        break

summer_olympics_table = tables_data[0]
winter_olympics_table = tables_data[1]

# Extracting footnotes
notes_section = soup.find('ol', class_='references')
notes = notes_section.find_all('li') if notes_section else []
notes_text = [note.text.strip() for note in notes]

In [119]:
summer_olympics_table.sample(5)

,Année,Olympiade(Nº de l’olympiade[NB 1]),Ville hôte,Pays,Continent
35,2028,XXXIV,Los Angeles (3),États-Unis (5),Amérique du Nord (7)
19,1964,XVIII,Tokyo (1),Japon (1),Asie (1)
1,1900,II,Paris (1),France (1),Europe (2)
26,1992,XXV,Barcelone (1),Espagne (1),Europe (14)
16,1952,XV,Helsinki (1),Finlande (1),Europe (10)


In [120]:
# Récupérer le texte des notes
notes_text

['↑ Ce numéro compte ainsi le nombre de périodes de quatre ans qui séparent les Jeux d’une année donnée des premiers Jeux d’Athènes en 1896 (qui portent le numéro I)\xa0: la numérotation ne tient ainsi pas compte du fait que des Jeux ont été supprimés, comme en 1916, 1940 et 1944, en raison des deux guerres mondiales.',
 "↑ Jeux olympiques de 1906 ne sont plus reconnus comme officiels par le CIO et sont qualifiés d'intercalaires.",
 '↑ Les Jeux olympiques de 1916 ont été annulés à cause de la Première Guerre mondiale',
 "↑ a et b Les Jeux olympiques d'été de 1940 puis de 1944 ont été annulés à cause de la Seconde Guerre mondiale",
 "↑ Les Jeux olympiques d'été de 2020 ont été reportés en 2021, à la suite de la pandémie de Covid-19.",
 '↑ Ce numéro enregistre ainsi le numéro de l’édition des Jeux d’hiver\xa0: il est en conséquence indépendant du fait que des Jeux d’hiver ont été supprimés, comme cela a été le cas en 1940 et 1944, en raison de la seconde guerre mondiale, puisque ceux-ci 

---

<a name="1"></a>
# *Minor Correction of the Data Scrapping* 
---

In [121]:
# Extract text starting form the first uppercase letter
for i, notes in enumerate(notes_text):
    for j, char in enumerate(notes):
        if char.isupper():
            notes_text[i] = notes[j:]
            break

In [122]:
notes_text

['Ce numéro compte ainsi le nombre de périodes de quatre ans qui séparent les Jeux d’une année donnée des premiers Jeux d’Athènes en 1896 (qui portent le numéro I)\xa0: la numérotation ne tient ainsi pas compte du fait que des Jeux ont été supprimés, comme en 1916, 1940 et 1944, en raison des deux guerres mondiales.',
 "Jeux olympiques de 1906 ne sont plus reconnus comme officiels par le CIO et sont qualifiés d'intercalaires.",
 'Les Jeux olympiques de 1916 ont été annulés à cause de la Première Guerre mondiale',
 "Les Jeux olympiques d'été de 1940 puis de 1944 ont été annulés à cause de la Seconde Guerre mondiale",
 "Les Jeux olympiques d'été de 2020 ont été reportés en 2021, à la suite de la pandémie de Covid-19.",
 'Ce numéro enregistre ainsi le numéro de l’édition des Jeux d’hiver\xa0: il est en conséquence indépendant du fait que des Jeux d’hiver ont été supprimés, comme cela a été le cas en 1940 et 1944, en raison de la seconde guerre mondiale, puisque ceux-ci ne sont pas comptés

In [123]:
winter_olympics_table

,Année,N° (Nº de l’édition[NB 6]),Ville hôte,Pays,Continent
0,1924,I,Chamonix (1),France (1),Europe (1)
1,1928,II,Saint-Moritz (1),Suisse (1),Europe (2)
2,1932,III,Lake Placid (1),États-Unis (1),Amérique du Nord (1)
3,1936,IV,Garmisch-Partenkirchen (1),Allemagne (1),Europe (3)
4,1940,V (annulés[NB 7]),Sapporo,Japon,Asie
5,Garmisch-Partenkirchen,Allemagne,Europe,None,None
6,1944,V (annulés[NB 7]),Cortina d'Ampezzo,Italie,Europe
7,1948,V,Saint-Moritz (2),Suisse (2),Europe (4)
8,1952,VI,Oslo (1),Norvège (1),Europe (5)
9,1956,VII,Cortina d'Ampezzo (1),Italie (1),Europe (6)


In [124]:
# Renaming column for clarity
summer_olympics_table.rename(columns={summer_olympics_table.columns[1]: 'Olympiad'}, inplace=True)
winter_olympics_table.rename(columns={winter_olympics_table.columns[1]: 'Olympiad'}, inplace=True)

# Fixing the duplicate entry issue in row 13
summer_olympics_table.iloc[13] = summer_olympics_table.iloc[12].values.tolist()[:2] + summer_olympics_table.iloc[13].values.tolist()[:3]
winter_olympics_table.iloc[5] = summer_olympics_table.iloc[4].values.tolist()[:2] + summer_olympics_table.iloc[5].values.tolist()[:3]

# Dropping the unofficial recognition
summer_olympics_table.drop(3, axis=0, inplace=True)
winter_olympics_table.drop(5, axis=0, inplace=True)

---

<a name="1"></a>
# *Preprocessing of the data* 
---

In [125]:
# Loading of the map of the countries to iso codes
iso_to_country = json.loads(open('../data/iso_country.json').read())
# Inverse the mapping
country_to_iso = {v.lower(): k for k, v in iso_to_country.items()}
print(country_to_iso)

{'andorra': 'ad', 'united arab emirates': 'ae', 'afghanistan': 'af', 'antigua and barbuda': 'ag', 'anguilla': 'ai', 'albania': 'al', 'armenia': 'am', 'angola': 'ao', 'antarctica': 'aq', 'argentina': 'ar', 'american samoa': 'as', 'austria': 'at', 'australia': 'au', 'aruba': 'aw', 'åland islands': 'ax', 'azerbaijan': 'az', 'bosnia and herzegovina': 'ba', 'barbados': 'bb', 'bangladesh': 'bd', 'belgium': 'be', 'burkina faso': 'bf', 'bulgaria': 'bg', 'bahrain': 'bh', 'burundi': 'bi', 'benin': 'bj', 'saint barthélemy': 'bl', 'bermuda': 'bm', 'brunei': 'bn', 'bolivia': 'bo', 'caribbean netherlands': 'bq', 'brazil': 'br', 'bahamas': 'bs', 'bhutan': 'bt', 'bouvet island': 'bv', 'botswana': 'bw', 'belarus': 'by', 'belize': 'bz', 'canada': 'ca', 'cocos (keeling) islands': 'cc', 'dr congo': 'cd', 'central african republic': 'cf', 'republic of the congo': 'cg', 'switzerland': 'ch', "côte d'ivoire (ivory coast)": 'ci', 'cook islands': 'ck', 'chile': 'cl', 'cameroon': 'cm', 'china': 'cn', 'colombia':

In [126]:
#define a function that translates a word from French to English
def translate_french_to_english(word): 
    if word: 
        return GoogleTranslator(source='fr', target='en').translate(word).lower()
    else:
        return None 

In [127]:
def clean_olympic_data(
        data_olympic, 
        notes_text, 
        translate_french_to_english, 
        country_to_iso):
    data_olympic_clean = {}
    for _, row in data_olympic.iterrows():
        year = int(row['Année'])
        olympiad = row['Olympiad']

        if len(olympiad.split()) > 1:
            olympiad_number = row['Olympiad'].split()[0]
            notabene = translate_french_to_english(
                notes_text[int(row['Olympiad'].split()[-1][0]) - 1]
            )
        else:
            olympiad_number = row['Olympiad']
            notabene = None

        # Searching for patterns in the 'Host City' column
        city_match = (
            re.search(r'(.+?)\s*\((\d+)\)', row['Ville hôte'])
            if row['Ville hôte']
            else None
        )
        city = city_match.group(1) if city_match else row['Ville hôte']
        city = translate_french_to_english(city)
        city_count = int(city_match.group(2)) if city_match else None

        # Searching for patterns in the 'Country' column
        country_match = (
            re.search(r'(.+?)\s*\((\d+)\)', row['Pays'])
            if row['Pays']
            else None
        )
        country = country_match.group(1) if country_match else row['Pays']
        country = translate_french_to_english(country)
        iso = country_to_iso[country] if country in country_to_iso else None
        country_count = int(country_match.group(2)) if country_match else None

        # Searching for patterns in the 'Continent' column
        continent_match = (
            re.search(r'(.+?)\s*\((\d+)\)', row['Continent'])
            if row['Continent']
            else None
        )
        continent = continent_match.group(1) if continent_match else row['Continent']
        continent = translate_french_to_english(continent)
        continent_count = int(continent_match.group(2)) if continent_match else None

        # Storing the results in the dictionary
        data_olympic_clean[year] = {
            'Olympiad': olympiad_number,
            'City': city,
            'City_count': city_count,
            'Country': country,
            'Iso': iso,
            'Country_count': country_count,
            'Continent': continent,
            'Continent_count': continent_count,
            'Notabene': notabene
        }

    # Converting the dictionary to a DataFrame
    data_olympic_df = pd.DataFrame(data_olympic_clean).T
    return data_olympic_df


In [128]:
summer_olympics_df = clean_olympic_data(summer_olympics_table, notes_text, translate_french_to_english, country_to_iso)
summer_olympics_df

,Olympiad,City,City_count,Country,Iso,Country_count,Continent,Continent_count,Notabene
1896,I,athens,1,greece,gr,1,europe,1,None
1900,II,paris,1,france,fr,1,europe,2,None
1904,III,saint louis,1,united states,us,1,north america,1,None
1908,IV,london,1,united kingdom,gb,1,europe,3,None
1912,V,stockholm,1,sweden,se,1,europe,4,None
1916,VI,berlin,None,germany,de,None,europe,None,the 1916 olympics were canceled due to world w...
1920,VII,antwerp,1,belgium,be,1,europe,5,None
1924,VIII,paris,2,france,fr,2,europe,6,None
1928,IX,amsterdam,1,the netherlands,None,1,europe,7,None
1932,X,los angeles,1,united states,us,2,north america,2,None


In [129]:
winter_olympics_df = clean_olympic_data(winter_olympics_table, notes_text, translate_french_to_english, country_to_iso)
winter_olympics_df

,Olympiad,City,City_count,Country,Iso,Country_count,Continent,Continent_count,Notabene
1924,I,chamonix,1,france,fr,1,europe,1,None
1928,II,st. moritz,1,swiss,None,1,europe,2,None
1932,III,lake placid,1,united states,us,1,north america,1,None
1936,IV,garmisch-partenkirchen,1,germany,de,1,europe,3,None
1940,V,sapporo,None,japan,jp,None,asia,None,the 1940 and 1944 winter olympics were cancele...
1944,V,cortina d'ampezzo,None,italy,it,None,europe,None,the 1940 and 1944 winter olympics were cancele...
1948,V,st. moritz,2,swiss,None,2,europe,4,None
1952,VI,oslo,1,norway,no,1,europe,5,None
1956,VII,cortina d'ampezzo,1,italy,it,1,europe,6,None
1960,VIII,squaw valley,1,united states,us,2,north america,2,None


---

<a name="1"></a>
# *Saving the data  into JSON* 
---

In [25]:
# Lire les fichiers CSV
summer_olympics = pd.read_csv('../data/summer_olympics.csv')
winter_olympics = pd.read_csv('../data/winter_olympics.csv')

summer_olympics.rename(columns={summer_olympics.columns[0]: 'Year'}, inplace=True)
winter_olympics.rename(columns={winter_olympics.columns[0]: 'Year'}, inplace=True)

# Créer des structures de données JSON
summer_data = summer_olympics.to_dict(orient='records')
winter_data = winter_olympics.to_dict(orient='records')

# Fonction pour filtrer les NaN
def filter_nan(dct):
    return {key: value for key, value in dct.items() if pd.notna(value)}

# Écrire le fichier JSON pour les Jeux d'été
with open('../data/summer_olympics.json', 'w') as f:
    json.dump([filter_nan(record) for record in summer_data], f)

# Écrire le fichier JSON pour les Jeux d'hiver
with open('../data/winter_olympics.json', 'w') as f:
    json.dump([filter_nan(record) for record in winter_data], f)